# Retrieval Augmented Generation (RAG) Server

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download hkunlp/instructor-large \
    --revision 54e5ffb8d484de506e59443b07dc819fb15c7233 \
    --local-dir ~/.gai/models/instructor-large \
    --local-dir-use-symlinks False
```


---

## 2. Load test configuration

In [2]:
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

config = {
    "type": "rag",
    "generator_name": "instructor-sentencepiece",
    "chromadb": {
        "path": "rag/chromadb",
        "n_results": 3
    },
    "sqlite": {
        "path": "rag/gai-rag.db"
    },
    "model_path": "models/instructor-large",
    "device": "cuda",
    "chunks": {
        "size": 1000,
        "overlap": 100,
        "path": "chunks"
    },
    "module_name": "gai.rag.server.gai_rag",
    "class_name": "RAG",
    "init_args": [],
    "init_kwargs": {}
}


## 3. Load Model Test

In [3]:
# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()
    

Free memory: 3.75 GB

INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer


/home/kakkoii1337/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 3.92 seconds │ 3.92 seconds │
│ CPU Memory        │          - MB │      5.73 MB │      5.73 MB │
│ CUDA Memory       │       3.78 GB │     -0.00 GB │      3.78 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

Free memory: 3.78 GB

Free memory: 3.77 GB

## Indexing

In [4]:
from gai.rag.server.gai_rag import RAG

try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag = host.generator

        # Index
        doc_id = await rag.index_async(
            collection_name='demo',
            file_path="./pm_long_speech_2023.txt",
            file_type='txt',
            source="https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech",
            title="2023 National Day Rally Speech"
            )

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 1.63 seconds │ 1.63 seconds │
│ CPU Memory        │          - MB │      1.50 MB │      1.50 MB │
│ CUDA Memory       │       3.76 GB │      0.01 GB │      3.76 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     rag.index_document_header_async: request started. collection_name=demo file_path=./pm_long_speech_2023.txt title=2023 National Day Rally Speech source=https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech abstract=None authors=None publisher=None published_date=None comments=None keywords=None
DEBUG    rag.index_document_header_async: creating doc header with id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U.
DEBUG    rag.index_document_header_async: document_header created. id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U
INFO     rag.index_document_split_async: splitting chunks
INFO     rag.index_document_split_async: chunkgroup created. chunkgroup_id=d0d0b11d-7be3-4055-b927-637708c6627b
100%|██████████| 66/66 [00:00<00:00, 392.50it/s]
INFO     rag.index_document_split_async: chunks created. count=66
INFO     RAG.index_document_index_async: Start indexing...
0it [00:00, ?it/s]DEBUG    RAG.index_document_index_async: Indexed 1/66 chunk e54bd9b8-7f1e-4199-ae45-1861b37042fb

Free memory: 3.62 GB

In [5]:
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag=host.generator
        # Index
        result = rag.retrieve(collection_name="demo",query_texts="Who are the young seniors?")
        console.print(result)

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 2.24 seconds │ 2.24 seconds │
│ CPU Memory        │          - MB │      1.49 MB │      1.49 MB │
│ CUDA Memory       │       3.62 GB │      0.01 GB │      3.62 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     RAG.retrieve: Retrieving by query Who are the young seniors?...


[
    {
        'documents': 'Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. 
"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you 
will soon retire, or maybe you have recently retired.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'd0d0b11d-7be3-4055-b927-637708c6627b',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.09020859003067017,
        'ids': '6906ca9a-4363-42f1-8dc0-b5eb34f2e0b9'
    },
    {
        'documents': 'Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka 
Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to 
workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have 
also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. 
Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of
caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully 
independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who 
may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,
plus attend to their everyday needs. All this, while watching your own health, because you are not so youthful',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'd0d0b11d-7be3-4055-b927-637708c6627b',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11518943309783936,
        'ids': '4e0f8747-ad59-46bd-bcc5-db1050ad39c7'
    },
    {
        'documents': 'to their everyday needs. All this, while watching your own health, because you are not so 
youthful yourselves. Responsibilities on all these fronts multiply your burdens. So, as Young Seniors, you are 
understandably anxious about your retirement needs. Beyond the daily cost of living pressures, you know that 
retirement is creeping up on you. You wonder: Will I have enough to get by? Can I cope? But do not worry – the 
Government will help you. You will not be left behind.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'd0d0b11d-7be3-4055-b927-637708c6627b',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11691617965698242,
        'ids': 'e5a4a0bb-b9a7-4f9d-aae0-7a82d8383ea5'
    }
]

Free memory: 3.62 GB

## API

#### a) List Collections

In [1]:
%%bash
curl -s http://localhost:12036/gen/v1/rag/collections

{"collections":[]}

#### b) Delete collection

In [2]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo

{"detail":{"code":"collections_not_found","message":"Collection demo not found"}}

#### c) index

In [3]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'collection_name=demo' \
    -F 'file=@./pm_long_speech_2023.txt' \
    -F 'metadata={"source": "https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023#:~:text=COVID%2D19%20was%20the%20most,indomitable%20spirit%20of%20our%20nation."}'

{"document_id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","chunkgroup_id":"26236065-51aa-4ff9-86fc-0617e8b84e32","chunk_ids":["6112d07e-833e-47f2-9706-9342bae36397","2717e31a-cd07-4852-94ed-e56d9fa8944a","e0f7ffff-0c9b-4102-86ec-4bf0ea2f39da","d690d60a-debf-485b-826e-6008e9f297a0","5defcdee-63f6-4b84-9ca7-e172af8ce704","f76ff583-2c75-436e-98bc-1f0e5090f5ec","72c71430-5b39-4189-8b4d-ede785ff6a01","d6dfc0f8-06bf-42df-86a9-abc37cb21ccc","da5a7008-ba0d-46c4-bd8f-881ebb33e5f5","336716e6-a248-4fad-b060-fd4820a6d60e","ec257dde-d855-4dda-98af-2bc338feb72c","f3154ec3-ea9d-43b4-bfcb-7eeb6c462335","31550df2-ee7f-43f6-96f0-c050e4f88c60","820ab2d1-952e-41da-93ca-e0f868f304f5","a8d13267-717a-4b27-90fe-ab5aeb1e8c4d","6b6219f8-fe27-485c-86d0-29c2f0afedd8","78619f40-1942-4f7e-8045-91e1a3293765","0434707f-3046-47cf-9b6f-7c7a210237a4","30123a90-b65b-4321-8d68-6636ad09b76f","85977a46-6dd9-45ef-ab4a-7e73b6981460","d4dc7f47-bbbf-4972-9655-3c6f7687b29a","719dcbe6-52fc-48fe-a1af-ad4fc1cd0cc5","e678815a-77b

#### d) verify document exists

In [9]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/collection/demo/document/exists' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' 

{"document":{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":"txt","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023#:~:text=COVID%2D19%20was%20the%20most,indomitable%20spirit%20of%20our%20nation.","Abstract":null,"Authors":"","Title":"","Publisher":"","PublishedDate":null,"Comments":"","Keywords":"","CreatedAt":"2024-09-04T00:38:01.807577","UpdatedAt":"2024-09-04T00:38:01.807589","IsActive":true,"ChunkGroups":[{"Id":"22d4eaf4-80df-44a4-bfc5-b898e1d220bb","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","SplitAlgo":"recursive_split","ChunkCount":66,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/2ea20321b20640de96dbc67a470daebd"}]}}

#### e) retrieve

In [10]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/retrieve' \
    -s \
    -H "Content-Type: application/json" \
    -d '{"collection_name":"demo","query_texts":"Who are the young seniors?","n_results":4}'


{"retrieved":[{"documents":"Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. \"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired.","metadatas":{"Abstract":"","ChunkGroupId":"22d4eaf4-80df-44a4-bfc5-b898e1d220bb","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","Keywords":"","PublishedDate":"","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023#:~:text=COVID%2D19%20was%20the%20most,indomitable%20spirit%20of%20our%20nation.","Title":""},"distances":0.09020859003067017,"ids":"91292b06-fb6a-4c72-8103-2de614603d5f"},{"documents":"Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lif

g) list documents

In [11]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/documents'

{"documents":[{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":"txt","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023#:~:text=COVID%2D19%20was%20the%20most,indomitable%20spirit%20of%20our%20nation.","Abstract":null,"Authors":"","Title":"","Publisher":"","PublishedDate":null,"Comments":"","Keywords":"","CreatedAt":"2024-09-04T00:38:01.807577","UpdatedAt":"2024-09-04T00:38:01.807589","IsActive":true,"ChunkGroups":[{"Id":"22d4eaf4-80df-44a4-bfc5-b898e1d220bb","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","SplitAlgo":"recursive_split","ChunkCount":66,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/2ea20321b20640de96dbc67a470daebd"}]}]}

h) get document

In [12]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U'

{"document":{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":"txt","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023#:~:text=COVID%2D19%20was%20the%20most,indomitable%20spirit%20of%20our%20nation.","Abstract":null,"Authors":"","Title":"","Publisher":"","PublishedDate":null,"Comments":"","Keywords":"","CreatedAt":"2024-09-04T00:38:01.807577","UpdatedAt":"2024-09-04T00:38:01.807589","IsActive":true,"ChunkGroups":[{"Id":"22d4eaf4-80df-44a4-bfc5-b898e1d220bb","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","SplitAlgo":"recursive_split","ChunkCount":66,"ChunkSize":1000,"Overlap":100,"IsActive":true,"ChunksDir":"/tmp/chunks/2ea20321b20640de96dbc67a470daebd"}]}}

i) update document

In [13]:
%%bash
curl -X PUT \
    http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "Publisher": "ABC"
        }'


{"message":"Document updated successfully","document":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U"}

j) delete document

In [14]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U

{"message":"Document with id PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U deleted successfully"}

---
#### Example : Index PDF using Multi-Step Indexing

The purpose for multi-step is to support interactive status update so that the client can make use of the websocket manager to get the status of the indexing process in a step-by-step manner.

NOTE: The same is achievable using the single-step indexing as well as before.

```
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'collection_name=demo' \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'metadata={"source": "https://arxiv.org/abs/1706.03762"}'
```


##### Step 1: Index document header

In [15]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/step/header' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'collection_name=demo' \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'metadata={"source": "https://arxiv.org/abs/1706.03762"}'

{"Id":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","CollectionName":"demo","ByteSize":2215244,"FileName":"attention-is-all-you-need.pdf","FileType":"pdf","Source":"https://arxiv.org/abs/1706.03762","Abstract":null,"Authors":"","Title":"","Publisher":"","PublishedDate":null,"Comments":"","Keywords":"","CreatedAt":"2024-09-04T00:41:34.325705","UpdatedAt":"2024-09-04T00:41:34.325714","IsActive":true,"ChunkGroups":[]}

##### Step 2: Split document into a group of chunks

In [16]:
import json
import os

# Execute the curl command and capture the response
response = !curl -X POST 'http://localhost:12036/gen/v1/rag/step/split' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{\
            "collection_name": "demo",\
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",\
            "chunk_size": 1000,\
            "chunk_overlap": 100\
        }'

# Convert response to string, then load it as JSON
response_json = json.loads(''.join(response))

# Extract the Id
document_id = response_json["Id"]
print(document_id)

# Set environment variable
os.environ['DOCUMENT_ID'] = document_id


4a9d145d-afd3-4dbf-a553-5c0575fef2bd


##### Step 3: Index each chunk in the database

In [17]:
%%bash
echo $DOCUMENT_ID
curl -X POST 'http://localhost:12036/gen/v1/rag/step/index' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "collection_name": "demo",
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",
            "chunkgroup_id": "'$DOCUMENT_ID'"
        }'

4a9d145d-afd3-4dbf-a553-5c0575fef2bd


{"chunk_ids":["0b53beed-ac8f-4e09-bd0f-de9252c324a9","a1ea7c31-d87c-42fa-a41b-85fde32f2fac","5ba10f98-fa01-47ca-b354-da40657c1e4b","ef0cd7b7-b4ed-4202-86cd-619f3121f191","712b0685-c4d9-493b-b1c3-01ffaa3a9c1d","4911395b-6677-44af-a111-b93d4d69b3b8","7ecbe361-aead-4a7c-8f2f-9b76acdfe8f9","01446e3f-fb3c-4ac1-a8a8-70ef29edd682","f4176ae5-6597-441c-9b10-f01007fedc29","14285cfc-3855-405d-802c-da8edbeffe16","74b145d7-9239-4a64-ba2e-027dd8fb7424","5f455173-f13d-4d48-93e9-878bef00b175","95f0e06d-8512-47e4-8a0f-41f5b15b8403","9cd4669c-ec90-49f1-beb0-46b4139ec58d","4e321eaf-3083-47cf-9356-d689e69150fc","77f7cec6-5234-4836-ad7a-b91f2021a464","bb615aef-0a0f-48fc-8ad4-2d380fae35c9","25ecc93b-ce56-4f00-94df-f4c8f64d4297","c9b20563-cc4d-4198-9382-41ad9c12560c","4aae71fb-a443-4601-bc52-0b5f4e96610c","89fbcce9-ee99-4cc3-9065-cd18654f5f36","1f11f6e5-e54c-4ca7-9987-9170caec1562","32e6d09f-755f-4eed-8d86-c54819fb37a6","819ed087-5619-450f-b604-fdaf0ebc0500","40fde5dc-bcb0-4884-8de7-a4b2664b89d9","ad1b5392-b

---

## 5. Docker

**Instructions:** 

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: up**

#### Smoke Test

In [ ]:
%%bash
curl http://localhost:12036


**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: down**

### Debugging

a) Container must be started with "python -m debugpy --listen 0.0.0.0:5678 main.py"

b) Port 5678 must be opened.

c) Click on "Debug" in Tool bar

d) Select "Attach" > "Run and Debug"

e) Add a "breakpoint" in the code

f) Run the API test to see if it trigger the breakpoint.